In [ ]:
!pip install chromadb


In [ ]:
!pip install autogen

In [ ]:
pip install dask[dataframe]

In [ ]:
!pip install markdownify

In [ ]:
!pip install pypdf

In [ ]:

import json

import os
import chromadb

import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.retrieve_utils import TEXT_FORMATS

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST.json")

assert len(config_list) > 0
print("models to use : ",[config_list[i]["model"] for i in range (len(config_list))])

models to use :  ['gpt-4o-mini', 'gpt-4o']


In [ ]:
print("Accepted file format for 'docs_path : ")
print(TEXT_FORMATS)

Accepted file format for 'docs_path : 
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf', 'mdx']


In [ ]:
#we are creating the assistance agent with system message set as You are a helpful assistant. and the detailed instruction is given in gthe user message
assistant = AssistantAgent(
    name = "assistant",
    system_message="You are a helpful assistant",
    llm_config={
        "timeout" : 600,
        "cache_seed" : 42,
        "config_list" : config_list,
    },
)

#next we are creating the RetrievUserProxyAgent
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": [
            "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
            "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md",
        ],
        "chunk_token_size": 2000,  #size of text chunks
        "model": config_list[0]["model"],  #use for which model has been used for the configuration list
        "vector_db": "chroma",   #vectorised document is stored  in chroma
        "overwrite": False,  #existing collection shpuld not overwroitten when retrieving or storing data
        "get_or_create": True,    #system wil either use the existing collection or create one
    },
    code_execution_config=False,   # here the agent will not execute any code as part of process
)


In [ ]:
#always reset the assistant first
assistant.reset()

code_problem = "How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached."
chat_result = ragproxyagent.initiate_chat(
    assistant, message=ragproxyagent.message_generator, problem=code_problem, search_string="spark"
)

2025-01-09 15:45:29,131 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.


Trying to create collection.


2025-01-09 15:45:29,170 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 2 chunks.


VectorDB returns doc_ids:  [['bdfbc921']]
Adding content of doc bdfbc921 to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached.

Context is: # Integrate - Spark

FLAML has integrated Spark for distributed training. There are two main aspects of integration with Spark:

- Use Spark ML estimators for AutoML.
- Use Spark to run training in

VectorDB returns doc_ids:  [['bdfbc921']]
No more context, will terminate.
ragproxyagent (to assistant):

TERMINATE

--------------------------------------------------------------------------------


In [ ]:
#example 2
assistant.reset()

qa_problem = "Who is the author of FLAML ?"
chat_result = ragproxyagent.initiate_chat(assistant, message = ragproxyagent.message_generator , problem = qa_problem)

VectorDB returns doc_ids:  [['7968cf3c', 'bdfbc921']]
Adding content of doc 7968cf3c to context.
Adding content of doc bdfbc921 to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Who is the author of FLAML ?

Context is: # Research

For technical details, please check our research publications.

- [FLAML: A Fast and Lightweight AutoML Library](https://www.microsoft.com/en-us/research/publication/flaml-a-fast-and-lightweight-automl-library/). Chi Wang, Qingyun Wu, Markus Weimer, Erkang Z

In [ ]:
#example 3
#in this example we can also give the human inpuit method
assistant.reset()

ragproxyagent.human_input_mode = "ALWAYS"
code_problem = "How to build a time series forecasting model for stock price using FLAML ?"
chat_result = ragproxyagent.initiate_chat(assistant, message = ragproxyagent.message_generator , problem = code_problem)

VectorDB returns doc_ids:  [['bdfbc921', '7968cf3c']]
Adding content of doc bdfbc921 to context.
Adding content of doc 7968cf3c to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: How to build a time series forecasting model for stock price using FLAML ?

Context is: # Integrate - Spark

FLAML has integrated Spark for distributed training. There are two main aspects of integration with Spark:

- Use Spark ML estimators for AutoML.
- Use Spark to run training in parallel spark jobs.

## S